# Safeguarding with Gemini

## Overview

Large language models (LLMs) can translate language, summarize text, generate creative writing, generate code, power chatbots and virtual assistants, and complement search engines and recommendation systems. The incredible versatility of LLMs is also what makes it difficult to predict exactly what kinds of unintended or unforeseen outputs they might produce. 

Given these risks and complexities, the Gemini is designed with [Google's AI Principles](https://ai.google/responsibility/principles/) in mind. However, it is important for developers to understand and test their models to deploy safely and responsibly. To aid developers, Vertex AI Studio has built-in content filtering, safety ratings, and the ability to define safety filter thresholds that are right for their use cases and business.

For more information, see the [Google Cloud Generative AI documentation on Responsible AI](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/responsible-ai).

## Learning Objectives

In this notebook, you learn how to inspect the safety ratings returned from Gemini using the Python SDK and how to set a safety threshold to filter responses from Gemini.

The steps performed include:

- Call Gemini via Gen AI SDK and inspect safety ratings of the responses
- Define a threshold for filtering safety ratings according to your needs

## Getting Started


### Define Google Cloud

In [2]:
PROJECT_ID = !gcloud config get-value project  # noqa: E999
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1"

### Import libraries


In [3]:
from google import genai
from google.genai.types import (
    GenerateContentConfig,
    HarmBlockThreshold,
    HarmCategory,
    Part,
    SafetySetting,
)

### Setup GenerateContentConfig for Gemini


In [4]:
MODEL = "gemini-2.0-flash"
client = genai.Client(vertexai=True, location="us-central1")

# Set parameters to reduce variability in responses
generation_config = GenerateContentConfig(
    safety_settings=[
        SafetySetting(
            category=HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
            threshold=HarmBlockThreshold.BLOCK_NONE,
        ),
        SafetySetting(
            category=HarmCategory.HARM_CATEGORY_HARASSMENT,
            threshold=HarmBlockThreshold.BLOCK_NONE,
        ),
        SafetySetting(
            category=HarmCategory.HARM_CATEGORY_HATE_SPEECH,
            threshold=HarmBlockThreshold.BLOCK_NONE,
        ),
        SafetySetting(
            category=HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
            threshold=HarmBlockThreshold.BLOCK_NONE,
        ),
    ]
)

## Generate text and show safety ratings

Start by generating a pleasant-sounding text response using Gemini.

In [5]:
# Call Gemini
nice_prompt = "Say three nice things about me"
responses = client.models.generate_content_stream(
    model=MODEL, contents=nice_prompt, config=generation_config
)
for response in responses:
    print(response.text, end="")

Okay, here are three nice things about you:

1.  **You're curious and open to learning.** Asking questions and wanting to know what others think is a great quality and shows a willingness to engage with the world.
2.  **You are proactive in seeking connection.** Reaching out to ask for positive feedback indicates you value connection and want to build positive relationships.
3.  **You have a desire to hear and accept compliments.** This shows a healthy self-esteem and a willingness to receive positivity, which is a lovely trait.


#### Inspecting the safety ratings

Look at the `safety_ratings` of the streaming responses.

In [6]:
response.candidates[0].to_json_dict()

{'content': {'parts': [{'text': ' a willingness to receive positivity, which is a lovely trait.\n'}],
  'role': 'model'},
 'finish_reason': 'STOP',
 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH',
   'probability': 'NEGLIGIBLE',
   'probability_score': 3.3969993e-08,
   'severity': 'HARM_SEVERITY_NEGLIGIBLE',
   'severity_score': 0.008249432},
  {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT',
   'probability': 'NEGLIGIBLE',
   'probability_score': 5.54802e-09,
   'severity': 'HARM_SEVERITY_NEGLIGIBLE',
   'severity_score': 0.04705216},
  {'category': 'HARM_CATEGORY_HARASSMENT',
   'probability': 'NEGLIGIBLE',
   'probability_score': 2.0369129e-07,
   'severity': 'HARM_SEVERITY_NEGLIGIBLE',
   'severity_score': 0.012135446},
  {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT',
   'probability': 'NEGLIGIBLE',
   'probability_score': 4.945674e-10,
   'severity': 'HARM_SEVERITY_NEGLIGIBLE',
   'severity_score': 0.06571972}]}

#### Understanding the safety ratings: category and probability

You can see the safety ratings, including each `category` type and its associated `probability` label.

The `category` types include:

* Hate speech: `HARM_CATEGORY_HATE_SPEECH`
* Dangerous content: `HARM_CATEGORY_DANGEROUS_CONTENT`
* Harassment: `HARM_CATEGORY_HARASSMENT`
* Sexually explicit statements: `HARM_CATEGORY_SEXUALLY_EXPLICIT`

The `probability` labels are:

* `NEGLIGIBLE` - content has a negligible probability of being unsafe
* `LOW` - content has a low probability of being unsafe
* `MEDIUM` - content has a medium probability of being unsafe
* `HIGH` - content has a high probability of being unsafe

The `probability_score` means the probability score in [0,1] about each safety categoy.<br>
Here you should be seeing very low values.

Try a prompt that might trigger one of these categories:

In [7]:
impolite_prompt = "Write a list of 5 disrespectful things that I might say to the universe after stubbing my toe in the dark:"

response = client.models.generate_content(
    model=MODEL, contents=impolite_prompt, config=generation_config
)

response.candidates[0].to_json_dict()

{'content': {'parts': [{'text': 'Okay, here are 5 disrespectful things you might say to the universe after stubbing your toe in the dark, embracing the raw frustration and (hopefully humorous) lack of reverence:\n\n1.  "Oh, real mature, Universe. Real classy move there. Did you get your jollies arranging the furniture specifically to cripple me?"\n\n2.  "Is this how you entertain yourself, you cosmic sadist? Watching mortals blunder around in the dark? Get a hobby, you infinite void!"\n\n3.  "Thanks a bunch, you uncaring expanse of nothingness! My toe is throbbing, and my faith in your benevolence is lower than the temperature of deep space."\n\n4.  "I\'d like to speak to a manager. Seriously, this level of \'divine plan\' is unacceptable. I demand a refund on my existence!"\n\n5.  "You know what, Universe? I\'m going to bed. And I\'m *not* going to say \'goodnight.\' You can just sit there in your vast emptiness and think about what you\'ve done."\n'}],
  'role': 'model'},
 'avg_logpr

Although you may not be seeing higher probability category since Gemini it self does a great job handling potentially harmful prompt, you may observe the probability_score is higher than the previous prompt.

### Defining thresholds for safety ratings

You may want to adjust the default safety filter thresholds depending on your business policies or use case. The Gemini provides you a way to pass in a threshold for each category.

The list below shows the possible threshold labels:

* `BLOCK_ONLY_HIGH` - block when high probability of unsafe content is detected
* `BLOCK_MEDIUM_AND_ABOVE` - block when medium or high probablity of content is detected
* `BLOCK_LOW_AND_ABOVE` - block when low, medium, or high probability of unsafe content is detected
* `BLOCK_NONE` - always show, regardless of probability of unsafe content

#### Set safety thresholds
Below, the safety thresholds have been set to the most sensitive threshold: `BLOCK_LOW_AND_ABOVE`

In [8]:
generation_config = GenerateContentConfig(
    safety_settings=[
        SafetySetting(
            category=HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
            threshold=HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        ),
        SafetySetting(
            category=HarmCategory.HARM_CATEGORY_HARASSMENT,
            threshold=HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        ),
        SafetySetting(
            category=HarmCategory.HARM_CATEGORY_HATE_SPEECH,
            threshold=HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        ),
        SafetySetting(
            category=HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
            threshold=HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        ),
    ]
)

#### Test thresholds

Here you will reuse the impolite prompt from earlier together with the most sensitive safety threshold. It should block the response even with the `LOW` probability label.

Try multiple times until you see a blocked response.

In [9]:
impolite_prompt = "Write a list of 5 disrespectful things that I might say to the universe after stubbing my toe in the dark:"

response = client.models.generate_content(
    model=MODEL, contents=impolite_prompt, config=generation_config
)

response.candidates[0].to_json_dict()

{'content': {},
 'finish_reason': 'SAFETY',
 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH',
   'probability': 'NEGLIGIBLE',
   'probability_score': 1.1307999e-05,
   'severity': 'HARM_SEVERITY_NEGLIGIBLE'},
  {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT',
   'probability': 'NEGLIGIBLE',
   'probability_score': 3.5073757e-05,
   'severity': 'HARM_SEVERITY_NEGLIGIBLE',
   'severity_score': 0.116353795},
  {'blocked': True,
   'category': 'HARM_CATEGORY_HARASSMENT',
   'probability': 'LOW',
   'probability_score': 0.07025014,
   'severity': 'HARM_SEVERITY_LOW',
   'severity_score': 0.16438755},
  {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT',
   'probability': 'NEGLIGIBLE',
   'probability_score': 2.3261121e-07,
   'severity': 'HARM_SEVERITY_NEGLIGIBLE',
   'severity_score': 0.013580173}]}

This notebook is based on [Thu Ya Kyaw](https://github.com/iamthuya)'s work.<br>
https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/responsible-ai/gemini_safety_ratings.ipynb

Copyright 2024 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License